## An unsupervised approach to rank product reviews
### Jianwei Wu ; Bing Xu ; Sheng Li ; 2011

In [11]:
import nltk
import pandas as pd
import numpy as np
import MHR as mhr
import networkx as nx
from nltk import word_tokenize

In [12]:
def zeroFun(x): return 0
dfProducts = pd.read_csv('data/eletronic_sample_counts.csv.gz')
dfProducts['pageRank']=dfProducts.apply(zeroFun,axis=1)
dfProducts['hits']=dfProducts.apply(zeroFun,axis=1)
dfProducts.shape

(19756, 30)

In [13]:
def compute_tags(sentence):
    tags = []
    for token in nltk.pos_tag(nltk.word_tokenize(sentence.lower()), tagset = "universal"):
        if (token[1] in ['NOUN','VERB','ADJ']):
            tags.append(token[0])
    return tags

def compute_sentence(sentence1, sentence2):
    return (len(set(sentence1).intersection(sentence2)) > 0)

def compute_sentences_graph(sentences):
    sentences_count = len(sentences)
    matrix = np.zeros((sentences_count,sentences_count))

    for row in range(sentences_count):
        for col in range(sentences_count):
            matrix[row, col] = compute_sentence(sentences[row],sentences[col])

    return matrix

In [14]:
import time

# run your code


min_comments=30
min_votes=5

grouped=dfProducts[dfProducts['tot'].astype(int)>min_votes].groupby('asin')
total = len(grouped)
run = 1
performance=[]
for name, group in grouped:

    dffiltro = (dfProducts['asin']==name) & (dfProducts['tot'].astype(int)>min_votes)
    comments_count = dfProducts[dffiltro ]['tot'].values
    
    print("Run %d size %d for %d" % (run, len(comments_count), total))
    
    if ( (len(comments_count)>min_comments) ):
        
        start = time.time()
        
        sentences=[]
        
        print "computing"
        
        for t in dfProducts[dffiltro].T.to_dict().values():
            for s in nltk.sent_tokenize(t['reviewText']):
                sentences.append(compute_tags(s))   
                
        sentences_graph = compute_sentences_graph(sentences)
        nx_graph = nx.from_numpy_matrix(sentences_graph)
        pg_sentences = nx.pagerank_numpy(nx_graph)
        hits_sentences = nx.hits_numpy(nx_graph)

        count=0
        for t in dfProducts[dffiltro].T.to_dict().values():
            for s in nltk.sent_tokenize(t['reviewText']):
                #print t.values()
                dfProducts.loc[t.values()[0],'pageRank'] += float(pg_sentences[count])
                dfProducts.loc[t.values()[0],'hits'] += float(hits_sentences[1][count]) #authority score
                count += 1

            #break
        #break
        run += 1
        end = time.time()
        elapsed_phl = end - start
            
        df = dfProducts[dffiltro]
        start = time.time()
        df,ndcg_mhr = mhr.executeFromDf(df )
        end = time.time()
        elapsed_mhr = (end - start) #+ elapsed
        
        tempo={}
        tempo['product']=name
        tempo['tot_comments']=len(comments_count)
        tempo['elapsed_phl']=elapsed_phl
        tempo['elapsed_mhr']=elapsed_mhr
        performance.append(tempo)
        
        if run > 5:
            break
        run = run+1
    
pd.DataFrame(performance).to_csv('data/performance_PR_HS_LEN2.csv.gz', compression='gzip')

Run 1 size 40 for 383
computing
product=1400532655 ndcg=0.877750740057 (0.877750740057)
Run 3 size 33 for 383
computing
product=B00000J061 ndcg=0.91824167245 (0.91824167245)
Run 5 size 31 for 383
computing
product=B00001P4ZH ndcg=0.920164849108 (0.920164849108)


In [17]:
pd.DataFrame(performance)

,elapsed_mhr,elapsed_phl,product,tot_comments
0,2.531230,99.641754,1400532655,40
1,0.312215,48.997984,B00000J061,33
2,0.293213,40.551298,B00001P4ZH,31


In [16]:
#dfProducts.to_csv('data/eletronic_sample_counts.csv.gz', compression='gzip')